In [ ]:
import pandas as pd
import sqlite3

orders = pd.read_csv("orders.csv")

users = pd.read_json("users.json")

conn = sqlite3.connect("restaurants.db")

# with open("restaurants.sql", "r") as f:
#     sql_script = f.read()
# conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

merged = orders.merge(users, on="user_id", how="left")
final_df = merged.merge(restaurants, on="restaurant_id", how="left")

final_df.to_csv("final_food_delivery_dataset.csv", index=False)

final_df.groupby("city")["total_amount"].sum().sort_values(ascending=False)

print("Dataset merged")
print(final_df.head())

Dataset merged
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_2

In [20]:
import pandas as pd

csvfood = pd.read_csv("final_food_delivery_dataset.csv")


In [ ]:
#highest total revenue (total_amount) from Gold members
csvfood[csvfood["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [ ]:
#cuisine has the highest average order value across all orders
csvfood.groupby("cuisine")["total_amount"] \
       .mean() \
       .sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [ ]:
#distinct users placed orders worth more than ₹1000 in total (sum of all their orders)
user_total = csvfood.groupby("user_id")["total_amount"].sum()

high_spenders = user_total[user_total > 1000]

high_spenders.count()

np.int64(2544)

In [ ]:
#restaurant rating range generated the highest total revenue

bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0-3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"]

csvfood["rating_range"] = pd.cut(
    csvfood["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)


In [ ]:
csvfood.groupby("rating_range")["total_amount"] \
       .sum() \
       .sort_values(ascending=False)


/var/folders/n4/06svkh7535328_sl_9cwbdj80000gn/T/ipykernel_20016/1897655866.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  csvfood.groupby("rating_range")["total_amount"] \


rating_range
4.6-5.0    2197030.75
3.0-3.5    2136772.70
4.1-4.5    1960326.26
3.6-4.0    1717494.41
Name: total_amount, dtype: float64

In [ ]:
#Among Gold members, which city has the highest average order value

csvfood[csvfood["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [ ]:
#cuisine has the lowest number of distinct restaurants but still contributes significant revenue

summary = csvfood.groupby("cuisine").agg(
    restaurants=("restaurant_id","nunique"),
    revenue=("total_amount","sum")
)

summary.sort_values(["restaurants","revenue"], ascending=[True,False])


,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [ ]:
#percentage of total orders were placed by Gold members? (Rounded to nearest integer)

round((csvfood["membership"]=="Gold").mean()*100)

50

In [ ]:
#restaurant has the highest average order value but less than 20 total orders

options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

csvfood.groupby("restaurant_name_x")["total_amount"] \
       .agg(["count","mean"]) \
       .query("count < 20") \
       .loc[lambda x: x.index.isin(options)] \
       .sort_values("mean", ascending=False)


,count,mean
restaurant_name_x,,
Ruchi Foods Chinese,19,686.603158


In [ ]:
#combination contributes the highest revenue
csvfood.pivot_table(
    values="total_amount",
    index="membership",
    columns="cuisine",
    aggfunc="sum"
)


cuisine,Chinese,Indian,Italian,Mexican
membership,,,,
Gold,977713.74,979312.31,1005779.05,1012559.79
Regular,952790.91,992100.27,1018424.75,1072943.30


In [32]:
#quarter of the year is the total revenue highes

# date
csvfood["order_date"] = pd.to_datetime(csvfood["order_date"], dayfirst=True)

# quarter
csvfood["quarter"] = csvfood["order_date"].dt.to_period("Q")

# Revenue per quarter
csvfood.groupby("quarter")["total_amount"] \
       .sum() \
       .sort_values(ascending=False)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [33]:
#total orders were placed by users with Gold membership
(csvfood["membership"] == "Gold").sum()

np.int64(4987)

In [34]:
#total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city
round(
    csvfood[csvfood["city"] == "Hyderabad"]["total_amount"].sum()
)

1889367

In [37]:
#distinct users placed at least one order
csvfood["user_id"].nunique()


2883

In [38]:
#average order value (rounded to 2 decimals) for Gold members

round(
    csvfood[csvfood["membership"] == "Gold"]["total_amount"].mean(),
    2
)

np.float64(797.15)

In [39]:
#orders were placed for restaurants with rating ≥ 4.5
(csvfood["rating"] >= 4.5).sum()

np.int64(3374)

In [40]:
#orders were placed in the top revenue city among Gold members only
top_city = (
    csvfood[csvfood["membership"]=="Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

len(csvfood[(csvfood["membership"]=="Gold") & (csvfood["city"]==top_city)])

1337

In [41]:
#total rows

len(csvfood)


10000